# Webscraping

## Wat gaan we doen? 

- Leren over diverse scraping libraries 
- Leren over werken met Browser Developer Tools (onderzoeken wat je wilt scrapen)
- Werken met een aantal libraries vanuit Python
- Aanroepen van een URL: GET, POST request
- Parsen van data: (X)HTML en JSON

We gaan eerst een klein stukje introductie doen en daarna aan de slag. 



## Introductie webscraping

Dit notebook laat manieren zien hoe webpaginas te manipuleren zijn en hoe de data ervan te _scrapen_ is. 

> "_Scrapen is een computertechniek waarbij software wordt gebruikt om informatie van webpagina's te extraheren en al dan niet te analyseren._"  <sub>[wiki]<sub>

In feite maak je contact via Python om informatie op te halen van een externe informatiebron zoals een website of api om daar gegevens vandaan te halen. Deze gegevens worden vervolgens in de eigen omgeving opgeslagen. 
    
### Libraries
Python heeft een aantal [internet] tools die daarbij helpen.  
De meest gebruike zijn [urllib.request] en [html.parser].  
Deze libraries hebben een _low-level_ design waardoor het te customizen is.  
Hierdoor moet er een hoop code schrijven om de functies zo te maken dat het doel wordt bereikt.  
Gelukkig zijn er third-party libraries die een _high-level_ interface bieden.  
Python heeft ook de [json] module om objecten te encoden naar json en inkomende json te decoden.  

Er zijn veel Third-party libraries de gebruikt kunnen worden om data van het web te halen.  
Hieronder een kleine selectie waarvan we een paar in dit notebook gaat gebruiken.  
    
    

    
    
    

## Scrapen van een API

Het makelijkste om data op te vragen is doormiddel van een API. Veel websites hebben geen openbare API.

Toch kan het zo zijn dat de website een API gebruikt om data van de client naar de server te sturen.
Zo kan de website dynamisch _(zonder te refreshen)_ de data op de pagina vernieuwen.   
Hier wordt [XMLHTTPRequests (XHR)] voor gebruikt.

In de Developer Tools onder de tab Network zie je het dataverkeer en de daarbij horende HTTP request methods.  
Om goed te zien wat een website krijgt en verstuurd moet de optie Preserve log/ Persist Logs en de Disable Cache aan staan.  
Met deze opties aan behoudt de network tab alle logs en vraagt de website alle data op die anders werd onthouden in de Cache.  



[XMLHTTPRequests (XHR)]: https://nl.wikipedia.org/wiki/XMLHTTP
[requests]: https://docs.python-requests.org/en/master/index.html "Requests: HTTP for Humans"

#### Third-party libraries 
    
[requests] is de _high-level_ interface om [HTTP request methods] te versturen naar API's of websites.

[beautifulsoup4] wordt gebruikt om HTML of XML te parsen.  

[selenium], de meest bekende web-automation tool welk ook veel gebruikt wordt om websites te testen.  

[scrapy] een web-crawler framework voor Python met een hoop features.

[httpx] de _next-generation_ HTTP client voor Python.  
Als je bekend ben met [requests] dan is de overstap naar [httpx] klein.  
Ondersteunt HTTP2 en de Python [async] functionaliteit [requests] ondersteunt dit niet.


    
[internet]: https://docs.python.org/3/library/internet.html "Internet Protocols and Support"
[urllib.request]: https://docs.python.org/3/library/urllib.request.html "Extensible library for opening URLs"
[json]: https://docs.python.org/3/library/json.html "JSON encoder and decoder"
[html.parser]: https://docs.python.org/3/library/html.parser.html "Simple HTML and XHTML parser"

[httpx]: https://www.python-httpx.org/ "A next-generation HTTP client for Python"
[scrapy]: https://scrapy.org/ "Web Crawling Framework"
[selenium]: https://www.selenium.dev/documentation/en/ "selenium"
[beautifulsoup4]: https://beautiful-soup-4.readthedocs.io/en/latest/ "Beautiful Soup"
[requests]: https://docs.python-requests.org/en/master/index.html "Requests: HTTP for Humans"

[wiki]: https://nl.wikipedia.org/wiki/Scrapen
[HTTP request methods]: https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods "HTTP request methods"
[async]: https://docs.python.org/3/library/asyncio.html
    